<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2023<br>
<font color=3C99D size=5>
    Practical Assignment 1 - Search <br>
<font color=696880 size=4>
    Reza Soumi, Rouzbeh Pirayadi, Amir Mohammad Izadi

____

# Personal Data

In [1]:
# Set your student number
student_number = '98110402'
Name = 'Pooriya'
Last_Name = 'Safaei'

# Rules
- Make sure that all of your cells can be run perfectly. 

## Libraries

In [2]:
import pygame, time, csv
import numpy as np
from time import sleep
import heapq
from IPython.display import HTML

pygame 2.5.2 (SDL 2.28.3, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Uninformed & Informed Search (100 Points)

<font size=4>
Author: Reza Soumi
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

## Problem Definition

In This Problem, We want to investigate Maze Problem with the focus on implementing IDS and A-Star algorithm to solve it. 

The concept of the **Digital Twin** was first introduced in 1960 during NASA's Apollo project. In this project, many simulation engines was created in the ground where sensory data from the spacecraft was sent to simulation devices. The model would then update itself with this data and send response or action commands back to the spacecraft using advanced telecommunication protocols. This concept has recently gained popularity.

If you are interested in learning more about digital twin in the Apollo NASA project, you can read this [link](https://blogs.sw.siemens.com/simcenter/apollo-13-the-first-digital-twin/) which explains how these systems helped astronauts return home safely. Additionally, there is a [TEDx](https://www.ted.com/talks/karen_willcox_how_digital_twins_could_help_us_predict_the_future?language=en) talk dedicated to this concept that you may find interesting.

Now, let's consider a scenario where we have launched a simple spacecraft into space without any IoT devices. Unfortunately, our robot has become stuck in a maze. The starting position is at the top left corner and the destination is at the bottom right corner, as shown in the picture below.

<!-- ![Sample Image](Pics/map.PNG) -->
<div style="text-align:center">
  <img src="Pics/map.PNG" alt="alt text" style="width:50%;"/>
</div>

To solve this problem, we need to implement an intelligent digital twin for the spacecraft. This digital twin will gather information from the spacecraft and determine which action to take - up, left, down, or right. However, we cannot explore the entire map at once and can only take one step forward at a time.

## Code

You can use the following data structures, and feel free to use any other data structures as needed, but these should suffice.

In [3]:
###########################################
###### no need to change any thing ########
###########################################

class Node:
    def __init__(self, pos, parent, cost=None):
        self.x = pos[0]
        self.y = pos[1]
        self.parent = parent
        self.cost = cost

    def position(self):
        return (self.x, self.y)

class Queue:
    def __init__(self):
        self.list = []

    def push(self,item):
        self.list.insert(0,item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0

class PriorityQueue:
    """ O(1) access to the lowest-priority item """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

class Stack:
    def __init__(self):
        self.list = []

    def push(self,item):
        self.list.append(item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0

The code below takes a CSV file's path as the map address. In this CSV file, a value of 1 represents a passable block where the robot can navigate, while a value of 0 represents an impassable wall where the robot cannot go. There's no need to be concerned about Digital Twins at this point; we will implement them later.

In [4]:
###########################################
###### no need to change any thing ########
###########################################

def solve_maze(map_address, algorithm):
    grid = np.genfromtxt(map_address, delimiter=',', dtype=int)
    num_rows, num_columns = len(grid), len(grid[0])
    empty_block_count = np.count_nonzero(grid == 1)

    # Define start & goal positions
    start_pos = (0,0)
    goal_pos = (num_rows-1, num_columns-1)

    grid[0, 0] = 2
    grid[-1, -1] = 3

    grid_dim = (num_rows-1, num_columns-1)

    black, white, green, red, grey, blue, magenta = (0,0,0), (255, 255, 255), (50,205,50), (255,99,71), (211,211,211), (153,255,255), (255,0,255)
    idx_to_color = [black, white, green, red, blue, magenta]

    height = 15
    width = height
    margin = 1

    pygame.init()

    WINDOW_SIZE = [660, 660]
    screen = pygame.display.set_mode(WINDOW_SIZE)

    pygame.display.set_caption(f"{algorithm} Pathfinder. Solving: {map_address}")

    done = False
    run = False
    close = False

    clock = pygame.time.Clock()

    digital_twin = None

    if algorithm == "BFS":
        digital_twin = BFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "DFS":
        digital_twin = DFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "IDS":
        digital_twin = IDS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "A_Star":
        digital_twin = A_Star_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    else:
        return None
    
    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

            elif event.type == pygame.KEYDOWN:
                run = True
                start_t0 = time.time()

        screen.fill(grey)

        for row in range(num_rows):
            for column in range(num_columns):
                color = idx_to_color[grid[row, column]]
                pygame.draw.rect(screen, color, [(margin + width) * column + margin, (margin + height) * row + margin, width, height])

        clock.tick(60)
        pygame.display.flip()

        if run == True:
            sleep(0.01)
            solution, done, grid = digital_twin.update(grid=grid)

        if done == True:
            print(f"Total empty block numbers: {empty_block_count}")
            print(f"Explored block numbers: {np.count_nonzero(grid == 4)}")
            for pos in solution:
                grid[pos[0], pos[1]] = 5

            screen.fill(grey)

            for row in range(num_rows):
                for column in range(num_columns):
                    color = idx_to_color[grid[row, column]]
                    pygame.draw.rect(screen, color, [(margin + width) * column + margin, (margin + height) * row + margin, width, height])

            clock.tick(60)
            pygame.display.flip()

    print(f"Your maze solved with {algorithm} algorithm.")
    print(f"--- finished {time.time()-start_t0:.3f} s---")
    while not close:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                close = True

            elif event.type == pygame.KEYDOWN:
                close = True
    pygame.quit()

Below is an implementation of our digital twin using the Depth-First Search (DFS) algorithm. Take a moment to explore how it functions.

In [5]:
###########################################
###### no need to change any thing ########
###########################################

class DFS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.stack = Stack()
        self.stack.push(Node(pos=start_pos, parent=None))

    def get_successors(self, x, y):
        return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        curr_state = self.stack.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]: # 1: empty cell has not explored yet, 3: goal cell
                self.stack.push(Node(pos=step, parent=curr_state))

                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break
            
            grid[x, y] = 4 # visited

        return solution_path, done, grid

You can run the code below to see how our exploration works (it is recommended to run it on your local machine instead of Colab).

In [6]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="DFS")

Total empty block numbers: 881
Explored block numbers: 868
Your maze solved with DFS algorithm.
--- finished 18.455 s---


After running the code above and entering it on the screen, you should see the result, which resembles the GIF below.

In [15]:
gif_path = 'Pics/Animation.gif'
html_code = f'<div style="display: flex; justify-content: center;"> <img src="{gif_path}" alt="Your GIF" style="width: 40%;"/></div>'
HTML(html_code)

As you can see, all of the blocks have been explored using DFS algorithm. So, we need a better algorithm for this maze problem. You can think and devise a solution for this problem at this stage before proceeding.

Let's investigate other uninformed search methods. Your task now is to implement Breadth-First Search (BFS) algorithm to solve this maze problem. (Hint: You can simply just change the data structures used in the code provided above to convert DFS to BFS)

In [8]:
########################################################
######### implement BFS algorithm (10 Points) ##########
########################################################

class BFS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.queue = Queue()
        self.queue.push(Node(pos=start_pos, parent=None))

    def get_successors(self, x, y):
        return [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        curr_state = self.queue.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and \
                    grid[step[0], step[1]] in [1, 3]:  # 1: empty cell has not explored yet, 3: goal cell
                self.queue.push(Node(pos=step, parent=curr_state))

                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break

            grid[x, y] = 4  # visited

        return solution_path, done, grid

You can run the code below to see how BFS algorithm works for solving this maze problem.

In [9]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="BFS")

Total empty block numbers: 881
Explored block numbers: 358
Your maze solved with BFS algorithm.
--- finished 5.904 s---


It seems that BFS algorithm provides a good solution for our maze problem. Now, let's try implementing the Iterative Deepening Search (IDS) algorithm in the code below. The depth must increase incrementally (one per request to the digital twin) according to the game policy.

In [10]:
########################################################
######### implement IDS algorithm (15 Points) ##########
########################################################

class IDS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.max_depth = 0  # Maximum depth for the current iteration
        self.solution_path = []
        self.stack = Stack()
        self.stack.push(Node(pos=start_pos, parent=None, cost=0))
        self.visited = []

    def get_successors(self, x, y):
        return [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        if self.stack.isEmpty():
            self.max_depth += 1
            for state in self.visited:
                v_x, v_y = state.position()
                grid[v_x, v_y] = 1
            self.visited = []
            self.solution_path = []
            self.stack = Stack()
            self.stack.push(Node(pos=self.start_pos, parent=None, cost=0))

            return [], False, grid

        curr_state = self.stack.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        if not curr_state.cost == self.max_depth:
            for step in self.get_successors(x, y):
                if self.is_valid_cell(step) and \
                        grid[step[0], step[1]] in [1, 3]:  # 1: empty cell has not explored yet, 3: goal cell
                    self.stack.push(Node(pos=step, parent=curr_state, cost=curr_state.cost + 1))

                    if step == self.goal_pos:
                        done = True
                        solution_path = self.backtrack_solution(curr_state)
                        break

                grid[x, y] = 4  # visited
                self.visited.append(curr_state)

        return solution_path, done, grid

In [11]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="IDS")

Total empty block numbers: 881
Explored block numbers: 348
Your maze solved with IDS algorithm.
--- finished 832.049 s---


Question: 

So far, we have explored three uninformed search methods: DFS, BFS, and IDS. Briefly explain the advantages and disadvantages of each method in the context of the Maze Problem and compare their performance based on your observations (consider both time and memory). Suppose there is two destination points, address specific points based on this scenario.

Answer:
در الگوریتم bfs، با توجه به اینکه در هر مرحله نیاز است که تمام فرزندان یک راس که اکسپند میشود را نگه داریم، حافظه از اردر b به توان d است که در آن b تعداد فرزندان هر راس و d عمق راس مورد نظر است. از طرفی در الگوریتم dfs، تنها نیاز است که یک راس را نگه داریم و حافظه از O(d) است. در الگوریتم ids، حافظه مانند dfs خطی است. پس از لحاظ حافظه، الگوریتم bfs بسیاز ضعیف عمل میکند. اما در مورد زمان، در الگوریتم bfs، زمان اجرا از اردر b به توان d است، اما در الگوریتم dfs، زمان اجرا از میتواند بسیار بیشتر باشد و حتی در حالاتی که درخت نامناهی است هرگز به جواب نرسیم. در الگوریتم ids، زمان اجرا از bfs بیشتر است اما مزیت این الگوریتم این است که تضمین جواب الگوریتم BFS و حافظه کم الگوریتم DFS را همزمان برای ما فراهم میکنند.
در سناریویی که دو مقصد داشته باشیم، الگوریتم DFS چون در عمق حرکت میکند ممکن است ما را به مقصدی برساند که دورتر است اما دو الگوریتم BFS و IDS ما را به مقصدی خواهند رساند که فاصله کمتری از نقطه شروع دارد.

Question:

Provide a simple solution to improve the DFS algorithm's performance in terms of explored block numbers, Based on the game information.

Answer:
در الگوریتم dfs، اگر از یک راسی که اکسپند میشود، راس رندمی از فرزندان آن را انتخاب کنیم، ممکن است که به جواب نرسیم. اما اگر از یک راسی که اکسپند میشود، فرزندان آن را به یک ترتیب خاصی مرتب کنیم(مانند فاصله منهتنی از مقصد) و سپس راسی را برای ادامه مسیر عمق انتخاب کنیم که کمترین مقدار را دارد، میتوانیم الگوریتم DFS را کمی بهبود دهیم

Now, let's turn our attention to a second maze map, 'maze_1.csv'. solve this maze using the BFS algorithm.

In [12]:
solve_maze(map_address="mazes/maze_1.csv", algorithm="BFS")

Total empty block numbers: 881
Explored block numbers: 799
Your maze solved with BFS algorithm.
--- finished 13.201 s---


As you can see, BFS may not always perform optimally, as it can end up exploring a majority of the blocks. Let's proceed by implementing the A-Star algorithm in the next cell.

In [13]:
########################################################
######### implement A* algorithm (50 Points) ###########
########################################################

class A_Star_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.max_depth = 0  # Maximum depth for the current iteration
        self.solution_path = []
        self.queue = PriorityQueue()
        self.queue.push(Node(pos=start_pos, parent=None, cost=0), self.manhattan_distance(start_pos))
        self.visited = []

    def manhattan_distance(self, pos):
        return abs(pos[0] - self.goal_pos[0]) + abs(pos[1] - self.goal_pos[1])

    def get_successors(self, x, y):
        return [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        curr_state = self.queue.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and \
                    grid[step[0], step[1]] in [1, 3]:  # 1: empty cell has not explored yet, 3: goal cell
                self.queue.push(Node(pos=step, parent=curr_state, cost=curr_state.cost + 1),
                                self.manhattan_distance(step) + curr_state.cost + 1)

                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break

            grid[x, y] = 4  # visited
            self.visited.append(curr_state)

        return solution_path, done, grid

Now solve the 'maze_1.csv' with your A-star algorithm.

In [14]:
solve_maze(map_address="mazes/maze_1.csv", algorithm="A_Star")

Total empty block numbers: 881
Explored block numbers: 611
Your maze solved with A_Star algorithm.
--- finished 10.075 s---


Question:

Explain your heuristic and observation's result. Also, compare the A* method with Uniformed search in the context of the Maze problem.

Answer: 
هیورستیکی که در این جا انتخاب کرده ایم، فاصله منهتنی نقطه فعلی از مقصد است. چرا که در هر نقطه ای باشیم، حتی اگر در مسیر دیوار هم نباشد، به اندازه فاصله منهتنی مرحله دیگر باید حرکت کنیم تا به مقصد برسیم
از طرفی چون A* هیورستیک و هزینه تا به اینجای کار برای هر راس را نیز در نظر میگیرد، بنابر این اجازه نمیدهد که درخت جست و جو به صورت کاملا یکنواخت یا کاملا یکسو در یک عمق زیاد که به جواب نخواهیم رسید باز شود. بنابراین خانه های کمتری نسبت به جست و جوی یکنواخت مشاهده خواهند شد.